In [1]:
!git clone https://github.com/peppermenta/faceHack

Cloning into 'faceHack'...
remote: Enumerating objects: 6461, done.
remote: Counting objects: 100% (6461/6461), done.
remote: Compressing objects: 100% (6458/6458), done.
remote: Total 6461 (delta 2), reused 6458 (delta 1), pack-reused 0
Receiving objects: 100% (6461/6461), 61.98 MiB | 25.33 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [4]:
!pip3 install -r faceHack/requirements.txt

In [19]:
import torch
from PIL import Image
import numpy as np
import os      
import torchvision         

class FaceDataset(torch.utils.data.Dataset):
  def __init__(self,train=True,type='normal'):
    super().__init__()
    if(train):
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','train.csv'),delimiter=',',dtype=None,encoding='utf-8')
    else:
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','test.csv'),delimiter=',',dtype=None,encoding='utf-8')

    self.type=type

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    img_name = self.data[index][0]
    X = Image.open(os.path.join('faceHack','dataset','happy_images','{}.jpg'.format(img_name)))
    label = self.data[index][1]

    if(self.type=='normal'):
      if(label=='NOT smile'):
        y = 0
      else:
        y = 1
    else:
      if(label=='positive smile'):
        y = 1
      else:
        y = 0

    return X,y


In [20]:
facesDataset = FaceDataset()
a,b = facesDataset.__getitem__(20)
print(a)
transform = torchvision.transforms.Compose([
  torchvision.transforms.Resize(224)
  torchvision.transforms.RandomHorizontalFlip()
  torchvision.transforms.ToTensor(),
])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=133x133 at 0x7F8BB1F03690>
